In [ ]:
import pandas as pd
import itertools
import networkx as nx
from collections import Counter

df_tracks = pd.read_csv("DATA/spotify_genius_data.csv")

G_artists = nx.Graph()
artist_collab_counts = Counter()

for artists_str in df_tracks["artists"].dropna():
    artists = [a.strip() for a in artists_str.split(",")]
    for artist in artists:
        G_artists.add_node(artist)
    for a1, a2 in itertools.combinations(artists, 2):
        pair = tuple(sorted((a1, a2)))
        artist_collab_counts[pair] += 1


G_artists.add_weighted_edges_from( # add edges with weights
    [(a1, a2, w) for (a1, a2), w in artist_collab_counts.items()]
)


#############
# Calculate average popularity and first release year for each artist
pop_sum   = Counter()
pop_count = Counter()
first_release = {}

for _, row in df_tracks.iterrows():
    if pd.isna(row["artists"]):
        continue
    artists = [a.strip() for a in row["artists"].split(",")]
    pop = row.get("popularity", 0) or 0
    try:
        year = pd.to_datetime(row["release_date"]).year
    except:
        year = None
    for artist in artists:
        pop_sum[artist]   += pop
        pop_count[artist] += 1
        if year and (artist not in first_release or year < first_release[artist]):
            first_release[artist] = year
avg_popularity = {
    artist: pop_sum[artist] / pop_count[artist]
    for artist in pop_sum
}
###############

for artist in G_artists.nodes():
    G_artists.nodes[artist]["display_name"] = artist
    G_artists.nodes[artist]["total_popularity"] = avg_popularity.get(artist, 0)
    G_artists.nodes[artist]["first_release_year"] = first_release.get(artist)
    G_artists.nodes[artist]["num_collaborations"] = G_artists.degree(artist, weight="weight")

In [2]:
nx.info(G_artists)

AttributeError: module 'networkx' has no attribute 'info'